In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-price/house_price_train (1).csv
/kaggle/input/titanic-train/titanic_train (1).csv


In [2]:
import numpy as np
import pandas as pd
df= pd.read_csv("/kaggle/input/titanic-train/titanic_train (1).csv")
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
null_columns = df.columns[df.isnull().any()]
print("Columns with null values:", list(null_columns))

Columns with null values: ['Age', 'Cabin', 'Embarked']


In [4]:
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode().iloc[0])
df['Cabin'] = df['Cabin'].fillna('Unknown')

In [5]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Unknown,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Unknown,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Unknown,S


In [6]:
unknown_ratio = (df['Cabin'] == 'Unknown').mean()

print(f"Percentage of 'Unknown' in Cabin: {unknown_ratio:.2%}")

Percentage of 'Unknown' in Cabin: 77.10%


In [7]:
 df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Unknown,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Unknown,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Unknown,S


In [8]:
# Convert categorical columns to numeric
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
label_encoders = {}
for col in ['Sex', 'Embarked']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [9]:
# Select features and target
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = df[features]
y = df['Survived']

In [10]:
# Scale features to [0,1] for chi-square
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [11]:
#Chi-Square Feature Selection
from sklearn.feature_selection import SelectKBest, chi2
import pandas as pd

chi_selector = SelectKBest(score_func=chi2, k='all')
chi_selector.fit(X_scaled, y)

chi_scores = chi_selector.scores_

chi_results = pd.DataFrame({
    'Feature': features,
    'Chi2 Score': chi_scores
})

print("Chi-Square Feature Scores:\n", chi_results)


Chi-Square Feature Scores:
     Feature  Chi2 Score
0    Pclass   27.232933
1       Sex   92.702447
2       Age    0.275991
3     SibSp    0.322733
4     Parch    1.682917
5      Fare    8.819172
6  Embarked    5.101262


In [12]:
#Fisher Score Implementation & Application
import numpy as np

def fisher_score(X, y):
    classes = np.unique(y)
    scores = []
    for i in range(X.shape[1]):
        numerator = 0
        denominator = 0
        overall_mean = np.mean(X[:, i])
        for cls in classes:
            X_c = X[y == cls, i]
            mean_c = np.mean(X_c)
            var_c = np.var(X_c)
            n_c = len(X_c)
            numerator += n_c * (mean_c - overall_mean) ** 2
            denominator += n_c * var_c
        score = numerator / denominator if denominator != 0 else 0
        scores.append(score)
    return np.array(scores)

fisher_scores = fisher_score(X_scaled, y.values)

fisher_results = pd.DataFrame({
    'Feature': features,
    'Fisher Score': fisher_scores
})

print("\nFisher Score Feature Scores:\n", fisher_results)



Fisher Score Feature Scores:
     Feature  Fisher Score
0    Pclass      0.129394
1       Sex      0.418904
2       Age      0.004231
3     SibSp      0.001249
4     Parch      0.006708
5      Fare      0.070901
6  Embarked      0.028928


In [13]:
#Univariate feature selection (Or) ANOVA (Regression and Classification)
from sklearn.feature_selection import SelectKBest, f_classif
import pandas as pd

selector = SelectKBest(score_func=f_classif, k='all')
selector.fit(X_scaled, y)

scores = selector.scores_

result_df = pd.DataFrame({
    'Feature': features,
    'ANOVA F-score': scores
}).sort_values(by='ANOVA F-score', ascending=False)

print(result_df)


    Feature  ANOVA F-score
1       Sex     372.405724
0    Pclass     115.031272
5      Fare      63.030764
6  Embarked      25.717286
4     Parch       5.963464
2       Age       3.761528
3     SibSp       1.110572


In [14]:
#Select Top-k Features
selector_top3 = SelectKBest(score_func=f_classif, k=3)
X_new = selector_top3.fit_transform(X_scaled, y)

selected_features = [features[i] for i in selector_top3.get_support(indices=True)]
print("Top 3 selected features:", selected_features)


Top 3 selected features: ['Pclass', 'Sex', 'Fare']


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
import numpy as np

roc_auc_scores = {}

for feature in features:
    X_feat = X[[feature]].values  # use single feature
    model = LogisticRegression(max_iter=1000)
    
    # Use cross_val_score with scoring='roc_auc' for stable estimate
    scores = cross_val_score(model, X_feat, y, cv=5, scoring='roc_auc')
    roc_auc_scores[feature] = scores.mean()

# Convert to DataFrame for easy viewing
roc_auc_df = pd.DataFrame.from_dict(roc_auc_scores, orient='index', columns=['ROC-AUC'])
roc_auc_df = roc_auc_df.sort_values(by='ROC-AUC', ascending=False)

print(roc_auc_df)


           ROC-AUC
Sex       0.766818
Fare      0.689833
Pclass    0.682421
Embarked  0.574381
Parch     0.561409
Age       0.520457
SibSp     0.456460
